In [1]:
import sys
sys.path.append("../")

from transforms import *
import yaml
from types import SimpleNamespace

/home/kpintaric/lumen-irmas/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [53]:
CONFIG_PATH = "../config.yaml"

with open(CONFIG_PATH) as file:
  config = SimpleNamespace(**yaml.safe_load(file))
  
  print(config)

namespace(train_dir='./data/raw/IRMAS_Training_Data', valid_dir='./data/raw/IRMAS_Validation_Data', preprocess={'PreprocessPipeline': {'parameters': {'target_sr': 22050}}}, transforms={'Preemphasis': None, 'Spectrogram': {'parameters': {'sample_rate': 22050, 'n_mels': 128, 'hop_length': 160, 'n_fft': 800}}, 'LogTransform': None, 'PadCutToLength': {'parameters': {'max_length': 1024}}, 'Normalize': {'parameters': {'mean': -4.2677393, 'std': 9.1379948}}}, transforms_not_used=[{'type': 'FeatureExtractor', 'parameters': {'sr': 22050}}], signal_augments=None, spec_augments=None, batch_size=8, loss={'type': 'FocalLoss', 'parameters': {'alpha': 0.25, 'gamma': 2}}, optimizer={'type': 'AdamW', 'parameters': {'weight_decay': 0.0001}}, learning_rates={'base_model': {'parameters': {'lr': 1e-08}}, 'classifier': {'parameters': {'lr': 0.001}}}, num_accum=2, verbose=True, EPOCHS=4, metrics=['hamming_score', 'zero_one_score', 'mean_average_precision', 'mean_f1_score'], scheduler={'type': 'OneCycleLR', '

In [17]:
from torchvision.transforms import Compose

In [21]:
def init_obj(fn_dict, module, *args, **kwargs):
    
    if isinstance(fn_dict, dict):
        fn_name = list(fn_dict.keys())[0]
    if isinstance(fn_dict, str):
        fn_name = fn_dict
    
    fn = getattr(module, fn_name)
    fn_args = fn_dict[fn_name].get("parameters")
    
    if fn_args is not None:
        assert all([k not in fn_args for k in kwargs])
        fn_args.update(kwargs)
        
        return fn(*args, **fn_args)
    else:
        return fn(*args, **kwargs)

In [33]:
def init_transforms(config, module_name):
    
    if config is None:
        return None

    transforms = []
    for transform in config.keys():
        print(transform)
        fn = getattr(transform_module, transform)
        transforms.append(fn(config[transform]["parameters"]))
    
    return Compose(transforms)

In [56]:
def init_objs(fn_dict, module):
    
    if fn_dict is None:
        return None

    transforms = []
    for transform in fn_dict.keys():
        fn = getattr(module, transform)
        fn_args = fn_dict[transform]
        
        if fn_args is None:
            transforms.append(fn())
        else:
            fn_args = fn_args.get("parameters")
            transforms.append(fn(**fn_args))
    
    return transforms

In [76]:
fn_dict = config.preprocess

transforms = []
for transform in fn_dict.keys():
    fn = getattr(transform_module, transform)
    fn_args = fn_dict[transform]

    if fn_args is None:
            transforms.append(fn())
    else:
        fn_args = fn_args.get("parameters")
        transforms.append(fn(**fn_args))

In [86]:
init_obj(config.preprocess, transform_module)

PreprocessPipeline()

In [89]:
config.signal_augments is None

True

In [51]:
config.optimizer.get("parameters")

In [34]:
import transforms as transform_module
init_transforms(config.transforms, transforms)

Preemphasis


TypeError: 'NoneType' object is not subscriptable

In [16]:
import torch.optim as optim

In [15]:
from utils import init_obj

In [20]:
import loss

In [22]:
import models

In [24]:
model = models.RNN(128, 64, 3, 11)

In [30]:
opt = init_obj(config.optimizer, optim, model.parameters())